- Update: 2023.05.06

# 0. 配置参数

- `rockauto`：ROCKAUTO.COM

In [1]:
from gevent import monkey
monkey.patch_all()
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = = = = =

from selenium.webdriver import ChromeOptions, Chrome
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.by import By
import pandas as pd

# = = = = = = = = = = = = = = = = = =

chromedriver = '../../chromedriver.exe'

# = = = = = = = = = = = = = = = = = =

chrome_options = ChromeOptions()
chrome_options.add_experimental_option('excludeSwitches', ['enable-automation']) # Chrome正受到自动测试软件的控制
chrome_options.add_argument('--disable-blink-features=AutomationControlled') # window.navigator.webdriver=false
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36')
chrome_options.add_argument('--proxy-server=http://j933.kdltps.com:15818')
chrome_options.add_argument('blink-settings=imagesEnabled=false')

# = = = = = = = = = = = = = = = = = =

df_menu = pd.read_excel('./RFQ#Brake Master Cylinder050423.xlsx', header=0)
list_sku = df_menu['Reference Brand SKU'].to_list()
print('总数量：' + str(len(list_sku)))

# = = = = = = = = = = = = = = = = = =

df = pd.DataFrame(columns=['sku',
                           'OEM Number',
                           'Alternate Inventory Number',
                           'status'])

# = = = = = = = = = = = = = = = = = =

work = Queue()
for sku in list_sku:
    work.put_nowait(sku)

# = = = = = = = = = = = = = = = = = =

def crawler():
    global df
    
    while not work.empty():
        sku = work.get_nowait()
        url = 'https://www.rockauto.com/en/parts/dorman,' + str(sku)
        driver = Chrome(service=Service(chromedriver), options=chrome_options)
        time.sleep(0.3)
        
        for i in range(31):
            try:
                driver.get(url)

                # = = = = = = = = = = = = = = = = = =

                time.sleep(5)
                info = driver.find_element(By.XPATH, '//*[@class="ra-btn ra-btn-moreinfo"]').get_attribute('href')
                driver.get(info)
                            
                # = = = = = = = = = = = = = = = = = =
                
                time.sleep(5)
                
                dict_alternate = {'alternate_inventory_number': '', 'oem_number': ''}
                list_alternate = [j.text for j in driver.find_elements(By.XPATH, '//*[@aria-label="Alternate/OEM Part Number(s)"]')]
                for j in range(len(list_alternate)):
                    if '{Alternate Inventory Numbers: ' in list_alternate[j]:
                        dict_alternate['alternate_inventory_number'] = list_alternate[j].replace('{Alternate Inventory Numbers: ', '').replace('}', '').replace(', ', ';')
                    elif 'OEM / Interchange Numbers: ' in list_alternate[j]:
                        dict_alternate['oem_number'] = list_alternate[j].replace('OEM / Interchange Numbers: ', '').replace(', ', ';')

                # = = = = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'sku': [str(sku)],
                                        'OEM Number': [str(dict_alternate['oem_number'])],
                                        'Alternate Inventory Number': [str(dict_alternate['alternate_inventory_number'])],
                                        'status': ['ok']})
                    
                break
                
            except:
                if i == 30:
                    df_temp = pd.DataFrame({'sku': [str(sku)],
                                            'OEM Number': [''],
                                            'Alternate Inventory Number': [''],
                                            'status': ['error']})
                continue
                
            # = = = = = = = = = = = = = = = = = =
        
        df = pd.concat([df, df_temp], ignore_index=True).fillna('')
        print(url, ' [' + str(i) + '] - 剩余数量：' + str(work.qsize()))
        driver.close()

# = = = = = = = = = = = = = = = = = =

list_task = []
for _ in range(15):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df.to_excel('./Dorman_oe.xlsx', index=False)
print()
print('搞定')

总数量：8
https://www.rockauto.com/en/parts/dorman,M630912  [1] - 剩余数量：0
https://www.rockauto.com/en/parts/dorman,M630888  [0] - 剩余数量：0
https://www.rockauto.com/en/parts/dorman,M630974  [0] - 剩余数量：0
https://www.rockauto.com/en/parts/dorman,M630947  [0] - 剩余数量：0
https://www.rockauto.com/en/parts/dorman,M630823  [0] - 剩余数量：0
https://www.rockauto.com/en/parts/dorman,M134427  [0] - 剩余数量：0
https://www.rockauto.com/en/parts/dorman,M630270  [0] - 剩余数量：0
https://www.rockauto.com/en/parts/dorman,M630531  [0] - 剩余数量：0

搞定


C:\Users\Lennon\AppData\Local\Programs\Python\Python311\Lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
